AIDS Mortality example
======================
This notebook illustrates an example present in the Simpact Cyan documentation.
There, it is explained that upon infection with HIV the survival time is calculated
as
$$ t_{\rm survival} = \frac{C}{V_{\rm sp}^{-k}} $$
Because of the start of antiretroviral treatment, or because of dropping out of
treatment, the set-point viral load $V_{\rm sp}$
of a person can go down or up again, and the simulations below show what happens
to the AIDS based time of death.

This is a very artificial example, meant to illustrate the reasoning in the
[reference documentation](http://research.edm.uhasselt.be/~jori/simpact/current/simpact_cyan.html#aidsmortality)

In [1]:
import os

os.environ['PYTHONPATH'] = '/home/_delicht/src/who-aids-prop/python'
os.environ['SIMPACT_DATA_DIR'] = '/home/_delicht/src/who-aids-prop/data/'
os.environ['PATH'] = f'${os.environ["PATH"]}:/home/_delicht/src/who-aids-prop/build'

In [2]:
import sys

sys.path.append("../../../python/")

In [3]:
# First, we'll load the Python interface to Simpact Cyan, as well as the 'math' library
import pysimpactcyan
import math

In [4]:
# Create an instance of the PySimpactCyan class, with which we can run Simpact Cyan simulations
simpact = pysimpactcyan.PySimpactCyan()

Setting data directory to /home/_delicht/src/who-aids-prop/data/


In the example from the documentation, survival times of 10 or 50 years are used, depending on treatment. The function below calculates which viral load corresponds to this for a fixed value of $C$ and $k$:
$$V_{\rm sp} = \left(\frac{C}{t}\right)^\left(-\frac{1}{k}\right) $$

In [5]:
def solveForVsp(t, C, k):
    return (float(C)/t)**(-1.0/k)

We then set some values for $C$ and $k$, set the time to live before treatment to 10 years, the time to live because of treatment to 50 years, and we calculate the corresponding $V_{\rm sp}$ values

In [6]:
t_before = 10.0
t_after = 50.0
C = 1000
k = -0.5
Vsp_before = solveForVsp(t_before, C, k)
Vsp_after = solveForVsp(t_after, C, k);

Infection only
--------------

For the first simulation we'll infect a population consisting of a single person
when the simulation starts. We'll make sure that treatment is not used, so if
everything works correctly, the person should die at a simulation time of 10 years.


In [7]:
cfg = { 
    # To focus on the AIDS mortality event we'll just use a population that consists
    # of one man
    "population.nummen": 1,
    "population.numwomen": 0,
    # By default, at the start of the simulation a number of people will be infected
    # with HIV. We set the fraction to 100%, so that the only member of our population
    # will certainly get infected when the simulation starts.
    "hivseed.fraction": 1,
    # By setting the age scale for the normal (non-AIDS) mortality event to something
    # very large and by setting the simulation time to something large as well, we can
    # be sure that the simulation will only stop when the one person in it dies from
    # AIDS
    "mortality.normal.weibull.scale": 1000, 
    "population.simtime": 1000,
    # Here, we set the values of the C and k parameters that we defined earlier
    "mortality.aids.survtime.C": C,
    "mortality.aids.survtime.k": k,
    # To make sure that upon infection the person gets the value of Vsp that we want,
    # we use the 'usealternativeseeddist' setting which can be used to specify a distribution
    # for Vsp that can is used when seeding. This distribution should provide the Vsp value
    # on a log10 scale. Since we want one specific value, we'll use the 'fixed' distribution.
    "person.vsp.model.logdist2d.usealternativeseeddist": "yes",
    "person.vsp.model.logdist2d.alternativeseed.dist.type": "fixed",
    "person.vsp.model.logdist2d.alternativeseed.dist.fixed.value": math.log10(Vsp_before),
    
    # Because we don't want treatment in this example, we could in principle just give
    # the diagnosis event a very low hazard. But to make it easier for the following
    # examples we'll actually use a very high hazard for diagnosis so it will happen
    # very shortly after infection
    "diagnosis.baseline": 100,
    # To make sure that no treatment is performed, we'll set the threshold for treatment
    # to zero
    "monitoring.cd4.threshold": 0,
    # On the other hand, thinking ahead again, when a person does get offered treatment
    # we'd like to make sure that he accepts it. This is done by setting the ART
    # accept threshold to 100%, using a 'fixed' distribution again.
    "person.art.accept.threshold.dist.type": "fixed",
    "person.art.accept.threshold.dist.fixed.value": 1,
    # Here we set the interval for the monitoring event to one year.
    "monitoring.interval.piecewise.left": 1,
    "monitoring.interval.piecewise.right": 1,
    "monitoring.interval.piecewise.cd4s": 500,
    "monitoring.interval.piecewise.times": 1,
    # When a monitoring event is triggered and the person's CD4 count is below the
    # threshold, he will be offered treatment. We've already made sure that the person
    # will accept the treatment, and here we specify that treatment should alter the
    # Vsp value to Vsp_after. This is not actually used in this first example, but
    # will be in the next two.
    "monitoring.fraction.log_viralload": math.log10(Vsp_after)/math.log10(Vsp_before),
    "vmmc.enabled": "no",
    "EventVMMC.m_vmmcprobDist.dist.type": "uniform",
    "EventVMMC.m_vmmcprobDist.dist.uniform.max":  1,
    "EventVMMC.m_vmmcprobDist.dist.uniform.min": 0
}


In [8]:
# Now we'll execute the Simpact Cyan simulation with these settings. The `run` function returns
# an object which contains paths to the output files.
r = simpact.run(cfg, "/tmp/simptest")
r

Using identifier 'simpact-cyan-2024-05-01-17-01-19_223531_fspaRMzO-'
Results will be stored in directory '/tmp/simptest'
Running simpact executable 'simpact-cyan-release' ...
Done.

# read seed from /dev/urandom
# Rng engine mt19937
# Using seed 211204832
# Performing extra check on read configuration parameters
# WARNING: ignoring consistency check for config key population.agedistfile (config value is '/home/_delicht/src/who-aids-prop/data/sa_2003.csv')
# mNRM: using advanced algorithm
# Release version
# Simpact version is: 1.0.0
# Error running simulation: No next scheduled event found: No event found
# Current simulation time is 10
# Number of events executed is 17
# Started with 1 people, ending with 0 (difference is -1)


{'logevents': '/tmp/simptest/simpact-cyan-2024-05-01-17-01-19_223531_fspaRMzO-eventlog.csv',
 'logpersons': '/tmp/simptest/simpact-cyan-2024-05-01-17-01-19_223531_fspaRMzO-personlog.csv',
 'logrelations': '/tmp/simptest/simpact-cyan-2024-05-01-17-01-19_223531_fspaRMzO-relationlog.csv',
 'logtreatments': '/tmp/simptest/simpact-cyan-2024-05-01-17-01-19_223531_fspaRMzO-treatmentlog.csv',
 'logsettings': '/tmp/simptest/simpact-cyan-2024-05-01-17-01-19_223531_fspaRMzO-settingslog.csv',
 'loglocation': '/tmp/simptest/simpact-cyan-2024-05-01-17-01-19_223531_fspaRMzO-locationlog.csv',
 'logviralloadhiv': '/tmp/simptest/simpact-cyan-2024-05-01-17-01-19_223531_fspaRMzO-hivviralloadlog.csv',
 'configfile': '/tmp/simptest/simpact-cyan-2024-05-01-17-01-19_223531_fspaRMzO-config.txt',
 'outputfile': '/tmp/simptest/simpact-cyan-2024-05-01-17-01-19_223531_fspaRMzO-output.txt',
 'id': 'simpact-cyan-2024-05-01-17-01-19_223531_fspaRMzO-',
 'agedistfile': '/home/_delicht/src/who-aids-prop/data/sa_2003.csv

In [9]:
# We'll display the log of all events which is stored in the file specified by `logevents`.
print(open(r["logevents"]).read())

0.0000000000,HIV seeding,(none),-1,-1,-1.0000000000,(none),-1,-1,-1.0000000000
0.0000000000,diagnosis,man_1,1,0,13.7449714707,(none),-1,-1,-1.0000000000
0.0000853700,monitoring,man_1,1,0,13.7450568407,(none),-1,-1,-1.0000000000,CD4,990.416
0.2508866069,chronicstage,man_1,1,0,13.9958580775,(none),-1,-1,-1.0000000000
1.0000853700,monitoring,man_1,1,0,14.7450568407,(none),-1,-1,-1.0000000000,CD4,891.719
1.2550285293,debut,man_1,1,0,15.0000000000,(none),-1,-1,-1.0000000000
2.0000853700,monitoring,man_1,1,0,15.7450568407,(none),-1,-1,-1.0000000000,CD4,793.022
3.0000853700,monitoring,man_1,1,0,16.7450568407,(none),-1,-1,-1.0000000000,CD4,694.324
4.0000853700,monitoring,man_1,1,0,17.7450568407,(none),-1,-1,-1.0000000000,CD4,595.627
5.0000853700,monitoring,man_1,1,0,18.7450568407,(none),-1,-1,-1.0000000000,CD4,496.93
6.0000853700,monitoring,man_1,1,0,19.7450568407,(none),-1,-1,-1.0000000000,CD4,398.232
7.0000853700,monitoring,man_1,1,0,20.7450568407,(none),-1,-1,-1.0000000000,CD4,299.535
8.000

Each line starts with the time an event took place, followed by the name of the event and
some additional information. An overview:

 - At the start of the simulation, the HIV seeding event is executed which causes the only
   person in the population to get infected.
 - Because of the high baseline value for the diagnosis hazard, this man will get diagnosed
   immediately afterwards. At this point, a monitoring event is also scheduled a very short
   time later.
 - When the monitoring event is triggered, no treatment will be offered because we set the
   CD4 threshold to a very low value. But we keep track of the disease progression by
   scheduling monitoring events on a yearly basis.
 - Slightly later than three months after the infection (some randomness is added), the
   chronic stage event will take place.
 - One year and three months before the AIDS based time of death, the person advances to
   the AIDS stage, and six months before the AIDS related death to the final AIDS stage.
 - Finally, 10 years after the infection was started, the person dies from AIDS

Treatment
---------

In the second example, we'll make sure the person is treated after one year. To do so,
we make use of the simulation intervention, by which we can change parameters of the
previous simulation at certain times.

In the previous simulation, no treatment occurred because the CD4 threshold was set to zero.
To make sure that the person will get treated during the monitoring event that occurs 
after the first simulation year has passed, we'll trigger a simulation intervention just
before that time and set the CD4 threshold to a very high value (so the person's CD4 will
be below that value)

In [10]:
intCfg = {
    # This change in configuration will take place one year into the simulation
    "time": 0.99, 
    # At that time the CD4 threshold is set to a very large value, so that during
    # the next monitoring event the person will receive treatment
    "monitoring.cd4.threshold": 100000,
    # We're not interested in any more monitoring events, so we'll set the interval
    # to 1000 years
    "monitoring.interval.piecewise.left": 1000,
    "monitoring.interval.piecewise.right": 1000,
    "monitoring.interval.piecewise.times": 1000,
    # In this example we do not want the person to drop out of treatment, so the
    # dropout interval is set to the fixed value of 1000 years
    "dropout.interval.dist.type": "fixed",
    "dropout.interval.dist.fixed.value": 1000,
}

# Then we run the simulation with the existing configuration and with the intervention
# configuration above
r = simpact.run(cfg, "/tmp/simptest", interventionConfig=[intCfg])

Using identifier 'simpact-cyan-2024-05-01-17-01-23_223531_ksN3KwW5-'
Results will be stored in directory '/tmp/simptest'
Running simpact executable 'simpact-cyan-release' ...
Done.

# read seed from /dev/urandom
# Rng engine mt19937
# Using seed 1917390844
# Performing extra check on read configuration parameters
# WARNING: ignoring consistency check for config key intervention.baseconfigname (config value is 'simpact-cyan-2024-05-01-17-01-23_223531_ksN3KwW5-interventionconfig_%.txt')
# WARNING: ignoring consistency check for config key intervention.fileids (config value is '1')
# WARNING: ignoring consistency check for config key population.agedistfile (config value is '/home/_delicht/src/who-aids-prop/data/sa_2003.csv')
# mNRM: using advanced algorithm
# Release version
# Simpact version is: 1.0.0
# Error running simulation: No next scheduled event found: No event found
# Current simulation time is 45.9996
# Number of events executed is 9
# Started with 1 people, ending with 0 (diffe

In [12]:
# Let's show the event log again
print(open(r["logevents"]).read())

0.0000000000,HIV seeding,(none),-1,-1,-1.0000000000,(none),-1,-1,-1.0000000000
0.0000000000,diagnosis,man_1,1,0,59.3925284531,(none),-1,-1,-1.0000000000
0.0001067150,monitoring,man_1,1,0,59.3926351681,(none),-1,-1,-1.0000000000,CD4,997.434
0.2516475892,chronicstage,man_1,1,0,59.6441760423,(none),-1,-1,-1.0000000000
0.9900000000,intervention,(none),-1,-1,-1.0000000000,(none),-1,-1,-1.0000000000
1.0001067150,monitoring,man_1,1,0,60.3926351681,(none),-1,-1,-1.0000000000,CD4,901.236
1.0001067150,(treatment),man_1,1,0,60.3926351681,(none),-1,-1,-1.0000000000
44.7495731399,aidsstage,man_1,1,0,104.1421015929,(none),-1,-1,-1.0000000000
45.4995731399,finalaidsstage,man_1,1,0,104.8921015929,(none),-1,-1,-1.0000000000
45.9995731399,aidsmortality,man_1,1,0,105.3921015929,(none),-1,-1,-1.0000000000,intreatment,1



When the monitoring event is triggered at the simulation time of one year, the CD4 count is now
below the (very high) threshold that we installed in the simulation intervention that happened
right before. This causes the man to receive treatment, which in turn lowers his Vsp value.

As you can see, because of this the man now continues to live for 45 more years. Also note that
again one year and three months before dying, the AIDS stage event is triggered, and six months
before dying, the final AIDS stage.

Treatment and dropout
---------------------
This final simulation corresponds to the example that was given in the documentation.
As in the previous one, the person will get treated after one year, but now he will
drop out of treatment ten years later again.

In [14]:
intCfg = { 
    # This change in configuration will take place one year into the simulation
    "time": 0.99,
    # At that time the CD4 threshold is set to a very large value, so that during
    # the next monitoring event the person will receive treatment
    "monitoring.cd4.threshold": 100000,
    # To make sure that the person drops out of treatment 10 years later, we 
    # fix this value
    "dropout.interval.dist.type": "fixed",
    "dropout.interval.dist.fixed.value": 10,
    # To prevent a person from being re-diagnoses after dropout, we'll set the
    # baselline value for this hazard to a very negative value (causes a very low
    # hazard)
    "diagnosis.baseline": -100,
    # We're not interested in any more monitoring events, so we'll set the interval
    # to 1000 years
    "monitoring.interval.piecewise.left": 1000,
    "monitoring.interval.piecewise.right": 1000,
    "monitoring.interval.piecewise.times": 1000,
}

r = simpact.run(cfg, "/tmp/simptest", interventionConfig=[intCfg])

Using identifier 'simpact-cyan-2024-05-01-17-01-45_223531_RdLpByOb-'
Results will be stored in directory '/tmp/simptest'
Running simpact executable 'simpact-cyan-release' ...
Done.

# read seed from /dev/urandom
# Rng engine mt19937
# Using seed 1136225407
# Performing extra check on read configuration parameters
# WARNING: ignoring consistency check for config key intervention.baseconfigname (config value is 'simpact-cyan-2024-05-01-17-01-45_223531_RdLpByOb-interventionconfig_%.txt')
# WARNING: ignoring consistency check for config key intervention.fileids (config value is '1')
# WARNING: ignoring consistency check for config key population.agedistfile (config value is '/home/_delicht/src/who-aids-prop/data/sa_2003.csv')
# mNRM: using advanced algorithm
# Release version
# Simpact version is: 1.0.0
# Error running simulation: No next scheduled event found: No event found
# Current simulation time is 18
# Number of events executed is 11
# Started with 1 people, ending with 0 (differenc

In [16]:
# Let's show the event log again
print(open(r["logevents"]).read())

0.0000000000,HIV seeding,(none),-1,-1,-1.0000000000,(none),-1,-1,-1.0000000000
0.0000000000,diagnosis,man_1,1,0,11.8387075042,(none),-1,-1,-1.0000000000
0.0000396466,monitoring,man_1,1,0,11.8387471508,(none),-1,-1,-1.0000000000,CD4,915.41
0.2501606690,chronicstage,man_1,1,0,12.0888681732,(none),-1,-1,-1.0000000000
0.9900000000,intervention,(none),-1,-1,-1.0000000000,(none),-1,-1,-1.0000000000
1.0000396466,monitoring,man_1,1,0,12.8387471508,(none),-1,-1,-1.0000000000,CD4,832.724
1.0000396466,(treatment),man_1,1,0,12.8387471508,(none),-1,-1,-1.0000000000
3.1612924958,debut,man_1,1,0,15.0000000000,(none),-1,-1,-1.0000000000
11.0000396466,dropout,man_1,1,0,22.8387471508,(none),-1,-1,-1.0000000000
16.7500000000,aidsstage,man_1,1,0,28.5887075042,(none),-1,-1,-1.0000000000
17.5000000000,finalaidsstage,man_1,1,0,29.3387075042,(none),-1,-1,-1.0000000000
18.0000000000,aidsmortality,man_1,1,0,29.8387075042,(none),-1,-1,-1.0000000000,intreatment,0



The start of the simulation is the same as before and one year after getting infected,
the man receives treatment. But now, ten years after that time, the man decides to
drop out of treatment which causes his $V_{\rm sp}$ value to go up again. Instead of
living for tens of years afterwards, after dropping out only seven more years remain
until the person dies of AIDS.